# Compare model outputs of AICORE DL Model 

### Imports 

In [1]:
import plotly.express as px
import pandas as pd
import os
import glob
from pathlib import Path

# workaround buggy autocomplete
%config Completer.use_jedi = False

### Settings 

In [18]:
#LOGDIR = Path('logs') / '0.3.1'
LOGDIR = Path(r'M:\p_aicore\initze\aicore-uc2_CV6\logs')
#SUB_REGEX = '*'
SUB_REGEX = '*CV6*'

In [19]:
subdirs = list(LOGDIR.glob(SUB_REGEX))

### Data Loading 

In [80]:
df_list_tr = []
df_list_v = []
for s in subdirs:
    basename = s.name
    df_tr = pd.read_csv(s / 'train.csv')
    df_tr['basename'] = basename[:20]
    df_tr['type'] = 'train'
    df_list_tr.append(df_tr)
    
    
    df_v = pd.read_csv(s / 'val.csv')
    df_v['basename'] = basename[:20]
    df_v['type'] = 'val'
    df_list_v.append(df_v)

df_train = pd.concat(df_list_tr)
df_val = pd.concat(df_list_v)

### Plots 

In [86]:
# put data into plottable shape
df_train_melt = df_train.melt(id_vars=['Epoch', 'basename', 'type'],value_vars=['IoU'])
df_val_melt = df_val.melt(id_vars=['Epoch', 'basename', 'type'],value_vars=['IoU'])

In [82]:
# merge
df_melt = pd.concat([df_train_melt, df_val_melt])

In [85]:
px.line(df_melt, x='Epoch' , y='value' , color='basename', line_group='type')

#### Plot mean values of configurations 

In [44]:
df_val_melt['ds'] = df_val_melt.apply(lambda x: x.basename.split('__')[0], axis=1)
mean_vals = df_val_melt.groupby(['Epoch', 'ds']).mean().reset_index(drop=False)

In [58]:
px.line(mean_vals, x='Epoch', y='value', color='ds')